In [47]:
import os
import numpy as np


                
def toQuery(rootdir,outtxt,folderpos=0):
    
    #
    
    outfile = open(outtxt,'w')
    
    for subdir, dirs, files in os.walk(rootdir):
        for i in range(len(files)):
            if i == folderpos:
                tempO = open(os.path.join(subdir, files[i]))
                contents = tempO.read()
                for i in range(len(contents)):
                    outfile.write(contents[i])

                    
                    
def toDb(rootdir,outfna):
    
    #
    
    outfile = open(outfna,'w')
    
    for subdir, dirs, files in os.walk(rootdir):
        for i in range(len(files)):
            tempO = open(os.path.join(subdir, files[i]))
            contents = tempO.read()
            for i in range(len(contents)):
                outfile.write(contents[i])

                

def blastnCount(inOut):
    
    #
    
    blastnOut = open(inOut)
    
    seqcount = 0 #number of found matches
    totcount = 0 #number of sequences found in query 
    
    line = blastnOut.readline()
    while line:
        if (line == 'Sequences producing significant alignments:                          (Bits)  Value\n'):
            seqcount = seqcount + 1
            totcount = totcount + 1
        if (line == '***** No hits found *****\n'):
            totcount = totcount + 1
        line = blastnOut.readline()
    blastnOut.close()
    print(seqcount)
    print(totcount)



def onlyHits(infile,outfile):
    
    #
    
    fullBlastn = open(infile)
    onlyHits = open(outfile,'w')
    
    newblock = []
    hitcheck = 0
    
    line = fullBlastn.readline()
    while line:
        if((line[0] == 'Q') and (line[5] == '=')):
            if(newblock and hitcheck):
                for i in range(len(newblock)):
                    onlyHits.write(newblock[i])
            newblock = []
            hitcheck = 0 
        if(line == 'Sequences producing significant alignments:                          (Bits)  Value\n'):
            hitcheck = 1
        newblock.append(line)
        line = fullBlastn.readline()
    if(hitcheck):
        for i in range(len(newblock)):
            outonlyhits.write(newblock[i])
        newblock = []
        hitcheck = 0
    fullBlastn.close()
            

def toFasta(infile,outfile):
    
    #
    
    onlyHits = open(infile)
    fasForm = open(outfile,'w')
    
    newline = ''
    scorecheck = 0
    
    line = onlyHits.readline()
    while line:
        if(line[1:6] == "Score"):
            scorecheck = scorecheck + 1
        elif((line[0]) == 'Q') and (line[5] == '='):
            scorecheck = 0
            if(newline):
                fasForm.write(newline)
                fasForm.write('\n')
                newline = ''
            fasForm.write('>')
            check1 = 0
            line = line[7:]
            while(check1 == 0):
                for i in range(len(line)):
                    if(line[i] != '\n'):
                        fasForm.write(line[i])
                    else:
                        fasForm.write(' ')
                line = onlyHits.readline()
                if(len(line)<=1):
                    check1 = 1
            fasForm.write('\n')
        elif((line[0] == 'Q') and (line[5] != '=')):
            if(scorecheck < 2):
                for i in range(len(line)):
                    if((line[i] == 'A' or line[i] == 'T') or (line[i] == 'C' or line[i] == 'G')):
                        newline = newline + line[i]
        line = onlyHits.readline()
    fasForm.write(newline)
    onlyHits.close()


def lengthAve(infile,lowcut=0,highcut=1):
    
    #takes in onlyHits to fasta file. lowcut,hightcut are bottom and top percentile cutoffs for counting average length
    #default parameters will average all sequences. 
    
    inf = open(infile)
    
    alllengths = []
    line = inf.readline()
    while line:
        if(line[0] != '>'):
            curcount = 0
            line = line[1:-1]
            alllengths.append(len(line))
        line = inf.readline()
    inf.close()
    
    alllengths.sort()
    numcount = 0
    lencount = 0
    for i in range(int(round(lowcut*len(alllengths))),int(round(highcut*len(alllengths)))):
        numcount = numcount + 1
        lencount = lencount + alllengths[i]
        
    print(lencount/numcount)
    print(alllengths)
    
    
def removeMarks(infile,outfile):
    
    #
    
    mothurAlign = open(infile)
    out = open(outfile,'w')
    
    line = mothurAlign.readline()
    while line:
        for i in range(len(line)):
            if((line[i] != '.') and (line[i] != '-')):
                out.write(line[i])
        line = mothurAlign.readline()
    mothurAlign.close()
    
    
def originalTags(infile,template,outfile):
    
    #
    
    inf = open(infile)
    tem = open(template)
    outf = open(outfile,'w')
    
    line = inf.readline()
    line = tem.readline()
    check = 1
    while(line):
        outf.write(line)
        check = check * -1
        if(check == -1):
            line = tem.readline()
            line = inf.readline()
        else:
            line = inf.readline()
            line = tem.readline()
        
    inf.close()
    tem.close()
    
def phylipFormat(infile,outfile,seqnum):
    
    #seqnum = '259\n' if there are 259 sequences
    
    megaCSV = open(infile)
    mothurPhylip = open(outfile,'w')
    
    mothurPhylip.write('259\n')
    
    line = megaCSV.readline()
    while line:
        for i in range(len(line)):
            if(line[i] == ','):
                mothurPhylip.write('\t')
            else:
                mothurPhylip.write(line[i])
        line = megaCSV.readline()
        if(line[0] == '\n'):
            line = ''
    megaCSV.close()

In [9]:
#16srRNA
toDb('16sRefs','16srRNAref.fna')

In [14]:
onlyHits('16srRNAout.txt','16srRNAoutOnlyHits.txt')

In [17]:
toFasta('16srRNAoutOnlyHits.txt','16srRNAout.fasta')

In [21]:
removeMarks('16srRNAout.align','16srRNAflipped.fasta')

In [5]:
phylipFormat('16srRNA.dist','phyl16srRNA.dist')

In [32]:
removeMarks('16srRNAgpo.align','16srRNAgpoflipped.fasta')

In [34]:
originalTags('16srRNAgpoflipped.fasta','16srRNAgpo.fasta','16srRNAgpoformflip.fasta')

In [4]:
#recA 
toDb('recArefs','recAref.fna')

In [66]:
blastnCount('recAout.txt')

246
15248


In [6]:
onlyHits('recAout.txt','recAoutOnlyHits.txt')

In [7]:
toFasta('recAoutOnlyHits.txt','recAout.fasta')

In [35]:
removeMarks('recAgpo.align','recAgpoflipped.fasta')

In [37]:
originalTags('recAgpoflipped.fasta','recAgpo.fasta','recAgpoformflip.fasta')

In [13]:
#glnII
toDb('glnIIrefs','glnIIref.fna')

In [18]:
blastnCount('glnIIout.txt')

218
15248


In [16]:
onlyHits('glnIIout.txt','glnIIoutOnlyHits.txt')

In [17]:
toFasta('glnIIoutOnlyHits.txt','glnIIout.fasta')

In [38]:
removeMarks('glnIIgpo.align','glnIIgpoflipped.fasta')

In [39]:
originalTags('glnIIgpoflipped.fasta','glnIIgpo.fasta','glnIIgpoformflip.fasta')

In [19]:
#gltA
toDb('gltArefs','gltAref.fna')

In [20]:
blastnCount('gltAout.txt')

50
15248


In [21]:
onlyHits('gltAout.txt','gltAoutOnlyHits.txt')

In [23]:
toFasta('gltAoutOnlyHits.txt','gltAout.fasta')

In [24]:
#dnaK
toDb('dnaKrefs','dnaKref.fna')

In [32]:
blastnCount('dnaKout.txt')

140
15248


In [26]:
onlyHits('dnaKout.txt','dnaKoutOnlyHits.txt')

In [27]:
toFasta('dnaKoutOnlyHits.txt','dnaKout.fasta')

In [28]:
#rpoA
toDb('rpoArefs','rpoAref.fna')

In [34]:
blastnCount('rpoAout.txt')

0
15248


In [3]:
#nifh
toDb('nifHrefs','nifHref.fna')

In [6]:
blastnCount('nifHout.txt')

101
15248


In [7]:
onlyHits('nifHout.txt','nifHoutOnlyHits.txt')

In [9]:
toFasta('nifHoutOnlyHits.txt','nifHout.fasta')

In [15]:
removeMarks('nifHout.align','nifHflipped.fasta')

In [28]:
originalTags('nifHflipped.fasta','nifHout.fasta','nifHformflip.fasta')

In [10]:
#nodC
toDb('nodCrefs','nodCref.fna')

In [12]:
blastnCount('nodCout.txt')

98
15248


In [13]:
onlyHits('nodCout.txt','nodCoutOnlyHits.txt')

In [14]:
toFasta('nodCoutOnlyHits.txt','nodCout.fasta')

In [29]:
removeMarks('nodCout.align','nodCflipped.fasta')

In [31]:
originalTags('nodCflipped.fasta','nodCout.fasta','nodCformflip.fasta')

In [4]:
#flip 4 genes
removeMarks('16srRNAgpo.align','16srRNAgpoflip.fasta')

In [3]:
removeMarks('temp11.fasta','16srRNAgpoMA.fasta')

In [2]:
blastnCount('rpoAout.txt')

0
15248


In [3]:
toDb('redo/16sRefs','redo/16srRNAref.fna')

In [17]:
blastnCount('redo/16srRNAout.txt')

309
15248


In [18]:
onlyHits('redo/16srRNAout.txt','redo/16srRNAoutOnlyHits.txt')

In [44]:
toFasta('redo/16srRNAoutOnlyHits.txt','redo/16srRNAout.fasta')

In [60]:
lengthAve('redo/16srRNAout.fasta',lowcut = 0.3,highcut = 0.95)


1481.3034825870648
[40, 45, 52, 52, 63, 65, 65, 71, 85, 87, 99, 108, 109, 116, 126, 130, 130, 134, 160, 167, 177, 220, 231, 242, 259, 259, 264, 266, 280, 309, 346, 421, 456, 466, 481, 562, 616, 663, 690, 721, 747, 929, 940, 964, 965, 994, 1120, 1132, 1151, 1160, 1253, 1275, 1283, 1299, 1300, 1310, 1354, 1370, 1381, 1386, 1386, 1386, 1400, 1405, 1405, 1405, 1405, 1459, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1472, 1474, 1474, 1474, 1476, 1476, 1478, 1478, 1479, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480, 1480,

In [46]:
toFasta('16srRNAoutOnlyHits.txt','16srRNAout.fasta')

In [61]:
lengthAve('16srRNAout.fasta',lowcut = 0.3,highcut = 0.95)

1280.372972972973
[56, 108, 207, 219, 242, 266, 274, 303, 453, 460, 480, 561, 616, 662, 683, 683, 687, 730, 732, 826, 926, 963, 992, 1059, 1059, 1152, 1160, 1161, 1161, 1246, 1271, 1272, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1273, 1275, 1275, 1275, 1275, 1277, 1277, 1277, 1277, 1277, 1277, 1277, 1277, 1277, 1277, 1277, 1277, 1277, 1277, 127

In [5]:
toDb('redo/recArefs','redo/recAref.fna')

In [67]:
blastnCount('redo/recAout.txt')

262
15248


In [68]:
onlyHits('redo/recAout.txt','redo/recAoutOnlyHits.txt')

In [69]:
toFasta('redo/recAoutOnlyHits.txt','redo/recAout.fasta')

In [76]:
lengthAve('redo/recAout.fasta',0.1,0.9)

1073.6238095238095
[987, 1024, 1027, 1027, 1027, 1027, 1027, 1027, 1027, 1027, 1027, 1044, 1048, 1051, 1051, 1054, 1054, 1054, 1054, 1054, 1054, 1055, 1055, 1055, 1055, 1055, 1055, 1056, 1057, 1057, 1057, 1057, 1057, 1057, 1057, 1057, 1057, 1057, 1058, 1058, 1058, 1058, 1058, 1059, 1059, 1059, 1059, 1059, 1059, 1059, 1059, 1059, 1060, 1060, 1062, 1062, 1062, 1062, 1062, 1062, 1063, 1063, 1063, 1063, 1063, 1063, 1063, 1063, 1063, 1065, 1065, 1065, 1066, 1066, 1066, 1066, 1068, 1068, 1068, 1068, 1068, 1068, 1068, 1068, 1069, 1069, 1069, 1069, 1069, 1069, 1069, 1069, 1069, 1069, 1069, 1069, 1069, 1069, 1069, 1069, 1069, 1069, 1069, 1069, 1070, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1071, 1072, 1072, 1072, 1072, 1072, 1072, 1072, 1072, 1072, 1072, 1073, 1073, 1073, 1074, 1074, 1075, 1075, 1077, 1077, 1077, 107

In [72]:
toFasta('recAoutOnlyHits.txt','recAout.fasta')

In [77]:
lengthAve('recAout.fasta',0.1,0.9)

885.4387755102041
[623, 623, 632, 664, 665, 673, 682, 682, 685, 767, 773, 775, 776, 776, 778, 778, 778, 778, 783, 794, 794, 796, 797, 797, 797, 797, 802, 813, 813, 814, 815, 821, 821, 821, 821, 821, 835, 835, 838, 838, 838, 838, 838, 838, 838, 838, 838, 838, 838, 838, 838, 838, 838, 838, 838, 838, 838, 838, 838, 838, 838, 838, 838, 838, 838, 838, 839, 839, 839, 839, 839, 840, 840, 840, 841, 841, 841, 841, 841, 841, 841, 841, 841, 841, 841, 841, 841, 841, 841, 841, 841, 841, 841, 841, 841, 841, 841, 842, 842, 842, 842, 842, 842, 842, 842, 842, 842, 842, 842, 842, 842, 842, 842, 842, 842, 842, 843, 843, 843, 843, 843, 843, 854, 858, 872, 874, 886, 895, 901, 901, 902, 902, 902, 902, 902, 902, 902, 902, 902, 902, 902, 905, 916, 916, 918, 919, 919, 919, 919, 919, 919, 919, 934, 940, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 946, 947, 947, 947, 947, 947, 947, 947, 9

In [79]:
toDb('redo/glnIIrefs','redo/glnIIref.fna')

In [80]:
blastnCount('redo/glnIIout.txt')

272
15248


In [82]:
blastnCount('glnIIout.txt')

218
15248


In [83]:
onlyHits('redo/glnIIout.txt','redo/glnIIoutOnlyHits.txt')

In [85]:
toFasta('redo/glnIIoutOnlyHits.txt','redo/glnIIout.fasta')

In [88]:
lengthAve('redo/glnIIout.fasta',0.2,0.95)

1030.5441176470588
[89, 146, 237, 245, 245, 358, 394, 394, 394, 419, 443, 528, 530, 578, 589, 753, 972, 975, 981, 982, 989, 990, 990, 990, 990, 990, 990, 990, 990, 990, 990, 1006, 1006, 1006, 1006, 1009, 1009, 1009, 1009, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 1012, 1013, 1013, 1013, 1013, 1015, 1015, 1015, 1016, 1025, 1026, 1026, 1030, 1030, 1031, 1032, 1032, 1032, 1033, 1033, 1033, 1033, 1033, 1033, 1033, 1033, 1033, 1035, 1036, 1036, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 1039, 1039, 1040, 1040, 1040, 1040, 1040, 1040, 1040, 1040, 1040, 1040

In [90]:
toFasta('glnIIoutOnlyHits.txt','glnIIout.fasta')

In [91]:
lengthAve('glnIIout.fasta',0.2,0.95)

1002.638036809816
[222, 222, 358, 380, 491, 589, 589, 753, 932, 936, 980, 980, 981, 981, 981, 981, 981, 994, 997, 997, 997, 997, 997, 997, 997, 997, 997, 998, 998, 998, 998, 998, 998, 998, 998, 998, 998, 998, 998, 998, 998, 998, 998, 998, 998, 998, 998, 998, 998, 998, 998, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 1001, 

In [92]:
toDb('redo/gltArefs','redo/gltAref.fna')

In [94]:
blastnCount('redo/gltAout.txt')
blastnCount('gltAout.txt')

271
15248
50
15248


In [97]:
onlyHits('redo/gltAout.txt','redo/gltAoutOnlyHits.txt')

In [99]:
toFasta('redo/gltAoutOnlyHits.txt','redo/gltAout.fasta')

In [107]:
lengthAve('redo/gltAout.fasta')

1255.2767527675276
[121, 197, 583, 840, 842, 928, 1152, 1165, 1167, 1167, 1167, 1174, 1180, 1180, 1202, 1204, 1204, 1205, 1206, 1216, 1216, 1216, 1216, 1218, 1218, 1218, 1220, 1220, 1220, 1220, 1220, 1220, 1220, 1220, 1220, 1220, 1220, 1220, 1220, 1220, 1220, 1220, 1220, 1220, 1220, 1220, 1220, 1222, 1222, 1223, 1224, 1225, 1225, 1225, 1225, 1225, 1228, 1229, 1229, 1229, 1236, 1236, 1237, 1245, 1246, 1257, 1259, 1259, 1259, 1262, 1262, 1268, 1269, 1269, 1269, 1269, 1269, 1275, 1277, 1280, 1285, 1285, 1285, 1285, 1285, 1285, 1285, 1285, 1285, 1285, 1285, 1285, 1285, 1285, 1285, 1285, 1285, 1285, 1285, 1285, 1285, 1285, 1285, 1285, 1285, 1286, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1287, 1289, 1289, 1289, 1289, 1289, 1289, 1289, 1289, 1289, 1289, 1289, 1289, 1289, 1289, 1289, 1289, 1289, 1289, 1289, 1289, 1289, 1289, 12

In [101]:
toFasta('gltAoutOnlyHits.txt','gltAout.fasta')

In [106]:
lengthAve('gltAout.fasta')

1117.5
[1072, 1082, 1082, 1082, 1082, 1086, 1087, 1087, 1087, 1087, 1087, 1087, 1087, 1087, 1087, 1087, 1087, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1120, 1127, 1127, 1129, 1129, 1129, 1131, 1131, 1132, 1133, 1144, 1150, 1150, 1150, 1150, 1150, 1150, 1151, 1186, 1203]


In [96]:
toDb('redo/dnaKrefs','redo/dnaKref.fna')

In [108]:
blastnCount('redo/dnaKout.txt')
blastnCount('dnaKout.txt')

264
15248
140
15248


In [110]:
onlyHits('redo/dnaKout.txt','redo/dnaKoutOnlyHits.txt')

In [111]:
toFasta('redo/dnaKoutOnlyHits.txt','redo/dnaKout.fasta')

In [112]:
lengthAve('redo/dnaKout.fasta')

1902.3560606060605
[1564, 1570, 1575, 1603, 1810, 1811, 1822, 1825, 1825, 1882, 1882, 1882, 1882, 1882, 1885, 1885, 1885, 1885, 1885, 1885, 1885, 1885, 1885, 1886, 1886, 1886, 1886, 1886, 1886, 1886, 1886, 1886, 1886, 1886, 1886, 1886, 1886, 1886, 1886, 1886, 1886, 1888, 1888, 1888, 1888, 1888, 1888, 1889, 1889, 1889, 1891, 1892, 1892, 1893, 1893, 1893, 1893, 1894, 1894, 1894, 1895, 1895, 1895, 1896, 1897, 1897, 1897, 1898, 1902, 1902, 1902, 1903, 1903, 1904, 1904, 1904, 1904, 1904, 1904, 1905, 1905, 1905, 1905, 1906, 1906, 1906, 1906, 1906, 1908, 1910, 1910, 1910, 1910, 1910, 1910, 1910, 1910, 1910, 1910, 1910, 1910, 1910, 1913, 1913, 1915, 1915, 1915, 1915, 1915, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 1916, 19

In [114]:
toFasta('dnaKoutOnlyHits.txt','dnaKout.fasta')

In [115]:
lengthAve('dnaKout.fasta')

1131.6857142857143
[377, 441, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1060, 1069, 1080, 1080, 1080, 1080, 1129, 1133, 1135, 1135, 1138, 1138, 1138, 1138, 1138, 1138, 1138, 1138, 1138, 1138, 1138, 1138, 1138, 1138, 1138, 1138, 1138, 1138, 1138, 1141, 1142, 1142, 1142, 1144, 1144, 1146, 1151, 1155, 1155, 1155, 1155, 1155, 1155, 1155, 1155, 1162, 1162, 1162, 1164, 1164, 1164, 1165, 1165, 1165, 1167, 1167, 1167, 1167, 1167, 1167, 1167, 1167, 1167, 1167, 1167, 1167, 1167, 1167, 1167, 1176, 1178, 1187, 1187, 1187, 1187, 1187, 1187, 1189, 1191, 1192, 1192, 1192, 1192, 1192, 1192, 1192, 1192, 1192, 1192, 1193, 1196, 1199, 1199, 1201, 1201, 1398, 1412, 1468, 1560, 1562]


In [116]:
toDb('redo/rpoArefs','redo/rpoAref.fna')

In [119]:
blastnCount('redo/rpoAout.txt')

263
15248


In [120]:
onlyHits('redo/rpoAout.txt','redo/rpoAoutOnlyHits.txt')

In [125]:
toFasta('redo/rpoAoutOnlyHits.txt','redo/rpoAout.fasta')

In [122]:
lengthAve('rpoAout.fasta')

1004.5779467680609
[497, 498, 929, 1000, 1000, 1003, 1003, 1003, 1003, 1003, 1003, 1003, 1003, 1003, 1003, 1006, 1006, 1006, 1006, 1006, 1006, 1006, 1006, 1006, 1006, 1006, 1006, 1006, 1006, 1006, 1006, 1006, 1006, 1006, 1006, 1007, 1007, 1007, 1007, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1008, 1009, 1009, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010, 1010,

In [123]:
#reformatting of nifH, nodC with new toFasta
toFasta('nifHoutOnlyHits.txt','nifHout.fasta')
toFasta('nodCoutOnlyHits.txt','nodCout.fasta')